In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import json 
import cv2 
from pa3_demo import load_obj_each_frame
from hungarian import hungarian_assignment, build_cost_matrix

BBOX_FILE = "frame_dict.json"
VIDEO_FILE = "commonwealth.mp4"
box_dict = load_obj_each_frame(BBOX_FILE)

In [5]:
frame_dict = box_dict.copy()

for frame_number in range(1, len(frame_dict)):
    
    frame1 = frame_dict[f"{frame_number-1}"]
    frame2 = frame_dict[f"{frame_number}"]
    cost_mat = build_cost_matrix(frame1, frame2)
    tracks = hungarian_assignment(cost_mat)
    # print("Number of cars in frame1: ", len(frame1))
    # print("Number of cars in frame2: ", len(frame2))
    if frame_number == 1:
        for i, box in enumerate(frame1):
            box["id"] = i+1
        
    frame1_id, frame2_id = np.where(tracks==1)
    # print("Frame number: ", frame_number)
    # print(frame1_id)
    for i, box in enumerate(frame2):
        box["id"] = int(frame2_id[i]+1)
    # print(frame2)
    # print()

In [6]:
def draw_object(object_dict, image):
    x_min, y_min, w, h, id= object_dict["x_min"], object_dict["y_min"], object_dict["width"], object_dict["height"], object_dict["id"]
    image = cv2.rectangle(image, (x_min, y_min), (x_min + w, y_min + h), (0, 255, 0), 2)
    image = cv2.putText(image, f"ID: {id}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

def draw_objects_in_video(video_file,frame_dict, output_file = "part_2_demo_test.mp4"):
    cap = cv2.VideoCapture(video_file)
    ok, image = cap.read()
    vidwrite = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
    while ok:
        ######!!!!#######
        image = cv2.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
        ######!!!!#######
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        objects = frame_dict.get(str(frame_number), [])
        for obj in objects: 
            image = draw_object(obj, image)
        vidwrite.write(image)
        ok, image = cap.read()
    vidwrite.release()
    cap.release()

draw_objects_in_video(VIDEO_FILE, frame_dict, output_file="part_2_demo.mp4")
dumped = json.dumps(frame_dict)

with open("part_2_frame_dict.json" , "w") as f:
    f.write(dumped)

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
